In [1]:
!pip install konlpy
!pip install nltk
!pip install wordcloud
!pip install kiwipiepy

import konlpy
import nltk
import wordcloud
from nltk.corpus import wordnet
from kiwipiepy import Kiwi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 28.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.17.0-py3-none-any.whl size=34845033 sha256=90807d34f6f69a78d3d82668a4cbf2130fc21072cb7a850899a376606916925f
  Stored in directory: /root/.cache/pip/wheels/90/bb/ae/15e1cf26d17183040bac268fbf8d1ca1f45f7f4daa0fb76b91
Successfully built kiwipiepy-model


In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/도배하자

/content/gdrive/MyDrive/도배하자


In [4]:
import pandas as pd
import numpy as np
import random
import os
import re
import warnings
import pickle

warnings.filterwarnings('ignore')
train = pd.read_csv('train.csv')

In [5]:
test = pd.read_csv('test.csv')

In [6]:
# CSV 파일 경로 설정
train_file_path = 'train.csv'
test_file_path = 'test.csv'

# CSV 파일 읽기
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

pre_train=train_data.copy()
pre_test=test_data.copy()


In [7]:
kiwi = Kiwi()

kiwi.add_user_word('면진', 'NNG')

True

In [8]:
columns = ['질문_1', '질문_2','답변_1', '답변_2', '답변_3', '답변_4', '답변_5']


In [9]:
# 텍스트를 형태소 분석 및 토큰화하는 함수
def tokenize_text(text):
    word_list = []
    result = kiwi.tokenize(text)
    for token in result:
        word_list.append(f'{token.form}/{token.tag}')  # 형태소/품사 형식으로 저장
    return ' '.join(word_list)  # 형태소를 공백으로 구분하여 반환

# 데이터프레임의 특정 열들을 형태소 분석 및 토큰화하는 함수
def tokenize_data(df, columns):
    for column in columns:
        # 형태소 분석 및 토큰화된 결과를 기존 데이터프레임에 추가
        df[column] = df[column].apply(lambda x: tokenize_text(x))
    return df


In [10]:
token_train=tokenize_data(pre_train,columns)

In [11]:
print(token_train)

            id                                               질문_1  \
0    TRAIN_000           면진/NNG 장치/NNG 가/JKS 뭐/NP 이/VCP 야/EF ?/SF   
1    TRAIN_001       내진/NNG 설계/NNG 의/JKG 종류/NNG 좀/MAG 알려주/VV 어/EF   
2    TRAIN_002  철골/NNG 구조/NNG 의/JKG 장점/NNG 이/JKS 뭐/NP 이/VCP 야/...   
3    TRAIN_003  철골/NNG 철근/NNG 콘크리트/NNG 구조/NNG 가/JKS 뭐/NP 이/VCP...   
4    TRAIN_004  철골/NNG 구조/NNG 는/JX 어떤/MM 방식/NNG 이/JKS 있/VA 어/E...   
..         ...                                                ...   
639  TRAIN_639  벽장/NNG 부위/NNG 결/NNG 로/JKB 의/JKG 원인/NNG 이/JKS 뭐...   
640  TRAIN_640  AD/SL ,/SP PD/SL 에/JKB 면/XR 하/XSV ᆫ/ETM 벽체/NNG...   
641  TRAIN_641  외벽/NNG 모서리/NNG 부위/NNG 에/JKB 결/NNG 로/JKB 가/VV 어...   
642  TRAIN_642  창호/NNG 결/NNG 로/JKB 의/JKG 대책/NNG 은/JX 뭐/NP 이/VC...   
643  TRAIN_643  AD/SL ,/SP PD/SL 에/JKB 면/XR 하/XSV ᆫ/ETM 벽체/NNG...   

                                                  질문_2 category  \
0    면진/NNG 장치/NNG 에/JKB 사용/NNG 되/XSV 는/ETM 주요/NNG ...     건축구조   
1    내진/NNG 설계/NNG 에/JKB 는/JX 어떤/MM 종

In [12]:
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.')
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX')
p3 = re.compile('[가-힣A-Za-z0-9]+/VV')
p4 = re.compile('[가-힣A-Za-z0-9]+/VX')

In [13]:
# 동사 원형 변환 함수
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        corpus.append(ori_sent)
    return corpus

In [14]:
for column in columns:
  token_train[column] = stemming_text(token_train[column].tolist())


In [15]:

for column in columns:
  print(token_train[column])


0               면진/NNG 장치/NNG 가/JKS 뭐/NP 이/VCP 야/EF ?/SF
1          내진/NNG 설계/NNG 의/JKG 종류/NNG 좀/MAG 알려주다/VV 어/EF
2      철골/NNG 구조/NNG 의/JKG 장점/NNG 이/JKS 뭐/NP 이/VCP 야/...
3      철골/NNG 철근/NNG 콘크리트/NNG 구조/NNG 가/JKS 뭐/NP 이/VCP...
4      철골/NNG 구조/NNG 는/JX 어떤/MM 방식/NNG 이/JKS 있/VA 어/E...
                             ...                        
639    벽장/NNG 부위/NNG 결/NNG 로/JKB 의/JKG 원인/NNG 이/JKS 뭐...
640    AD/SL ,/SP PD/SL 에/JKB 면/XR 하/XSV ᆫ/ETM 벽체/NNG...
641    외벽/NNG 모서리/NNG 부위/NNG 에/JKB 결/NNG 로/JKB 가다/VV ...
642    창호/NNG 결/NNG 로/JKB 의/JKG 대책/NNG 은/JX 뭐/NP 이/VC...
643    AD/SL ,/SP PD/SL 에/JKB 면/XR 하/XSV ᆫ/ETM 벽체/NNG...
Name: 질문_1, Length: 644, dtype: object
0      면진/NNG 장치/NNG 에/JKB 사용되다/VV 는/ETM 주요/NNG 기술/NN...
1      내진/NNG 설계/NNG 에/JKB 는/JX 어떤/MM 종류/NNG 가/JKS 있다...
2      철골/NNG 구조/NNG 의/JKG 장점/NNG 을/JKO 알려주다/VV 어/EF ...
3      철골/NNG 철근/NNG 콘크리트/NNG 구조/NNG 의/JKG 장점/NNG 과/J...
4      철골/NNG 구조/NNG 의/JKG 다양하다/VV ᆫ/ETM 방식/NNG 이/JKS...
                             ...                 

In [16]:
stopwords = ['ᆸ니다/EF', '데/NNB', '좀/MAG','및/MAG', '수/NNB', '등/NNB', '를/JKO', '을/JKO', '의/JKG', '가/JKS', '으로/JKB']
# 불용어 제거 함수 정의
additional_tags = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', '/JX', '/JC']

def remove_stopword_text(text):
  corpus = []
  for sent in text.str.split(" "):
      modi_sent = []
      for word in sent:
         if word[-3:] in additional_tags :
            continue
         if word not in stopwords :
              modi_sent.append(word)
      corpus.append(' '.join(modi_sent))
  return corpus


In [17]:
for col in columns:
    token_train[col] = remove_stopword_text(token_train[col])


In [18]:
print(token_train)

            id                                               질문_1  \
0    TRAIN_000                 면진/NNG 장치/NNG 뭐/NP 이/VCP 야/EF ?/SF   
1    TRAIN_001                  내진/NNG 설계/NNG 종류/NNG 알려주다/VV 어/EF   
2    TRAIN_002          철골/NNG 구조/NNG 장점/NNG 뭐/NP 이/VCP 야/EF ?/SF   
3    TRAIN_003  철골/NNG 철근/NNG 콘크리트/NNG 구조/NNG 뭐/NP 이/VCP 야/EF ...   
4    TRAIN_004          철골/NNG 구조/NNG 어떤/MM 방식/NNG 있/VA 어/EF ?/SF   
..         ...                                                ...   
639  TRAIN_639    벽장/NNG 부위/NNG 결/NNG 원인/NNG 뭐/NP 이/VCP 야/EF ?/SF   
640  TRAIN_640  AD/SL ,/SP PD/SL 면/XR 하/XSV ᆫ/ETM 벽체/NNG 결/NNG...   
641  TRAIN_641  외벽/NNG 모서리/NNG 부위/NNG 결/NNG 가다/VV 어/EC 발생하다/VV...   
642  TRAIN_642           창호/NNG 결/NNG 대책/NNG 뭐/NP 이/VCP 야/EF ?/SF   
643  TRAIN_643  AD/SL ,/SP PD/SL 면/XR 하/XSV ᆫ/ETM 벽체/NNG 결로/NN...   

                                                  질문_2 category  \
0    면진/NNG 장치/NNG 사용되다/VV 는/ETM 주요/NNG 기술/NNG 무엇/N...     건축구조   
1    내진/NNG 설계/NNG 어떤/MM 종류/NNG 있다/VV